<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/Bhavya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# tf.config.set_visible_devices(tf.config.experimental.list_physical_devices('GPU')[0], 'GPU')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/data.csv')

In [ ]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


In [ ]:
df.dropna(inplace=True)

In [ ]:
df1=df[:3000]

In [ ]:
df1.to_csv('data1.csv',index=False)

In [ ]:
data=pd.read_csv('data1.csv')

In [ ]:
data.shape

(3000, 6)

**Split the dataset into train and evaluate**

In [ ]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(data, test_size=0.2)

In [ ]:
train_data.shape

(2400, 6)

In [ ]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')


In [ ]:
import csv

# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [ ]:
# import csv

# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [ ]:
# Load the pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [ ]:
# Prepare your dataset by creating a TextDataset object
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128
)



/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Prepare your dataset by creating a TextDataset object
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128
)


In [ ]:
# pip install pytorch-lightning[callbacks]

In [ ]:
# pip install pytorch-lightning==1.2.7

In [ ]:
# pip install --upgrade pytorch-lightning


In [ ]:
# pip uninstall pytorch-lightning


In [ ]:
# pip freeze | grep pytorch-lightning


In [ ]:
pip install pytorch-lightning


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

# Set up the model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='checkpoint-{epoch:02d}',
    save_top_k=2,
    monitor='loss',
    mode='min',
    save_last=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset,
    callbacks=[checkpoint_callback]
)


In [ ]:
class CustomModelCheckpoint(ModelCheckpoint):
    def on_train_start(self, trainer, pl_module):
        super().on_train_start(trainer, pl_module)
        # Your custom code here

checkpoint_callback = CustomModelCheckpoint(
    dirpath='./checkpoints',
    filename='checkpoint-{epoch:02d}',
    save_top_k=2,
    monitor='loss',
    mode='min',
    save_last=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset,
    callbacks=[checkpoint_callback]
)


In [ ]:
from transformers import EarlyStoppingCallback, IntervalStrategy
from pytorch_lightning.callbacks import ModelCheckpoint

# Set up the trainer with the appropriate training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    save_total_limit=2,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
class CustomModelCheckpoint(ModelCheckpoint):
    def on_train_start(self, trainer, pl_module):
        super().on_train_start(trainer, pl_module)
        # Your custom code here


checkpoint_callback = CustomModelCheckpoint(
    dirpath='./checkpoints',
    filename='checkpoint-{epoch:02d}',
    save_top_k=2,
    monitor='loss',
    mode='min',
    save_last=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset,
    callbacks=[checkpoint_callback]
)


In [ ]:
# # Set up the trainer with the appropriate training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     learning_rate=5e-5,
#     per_device_train_batch_size=16,
#     num_train_epochs=5,
#     save_total_limit=2,
# )
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     data_collator=data_collator,
#     eval_dataset=eval_dataset
# )



In [ ]:
# Fine-tune the model on your dataset
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.592800,3.346953
2,3.289700,3.309886
3,3.157100,3.297071
4,3.089000,3.296507
5,3.043300,3.298493


TrainOutput(global_step=4615, training_loss=3.218702177004334, metrics={'train_runtime': 2783.1486, 'train_samples_per_second': 26.52, 'train_steps_per_second': 1.658, 'total_flos': 4821491220480000.0, 'train_loss': 3.218702177004334, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 3.298492670059204,
 'eval_runtime': 39.8979,
 'eval_samples_per_second': 92.937,
 'eval_steps_per_second': 11.63,
 'epoch': 5.0}

In [ ]:
trainer

In [ ]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch

In [ ]:
# model.to('cuda')a# Convert the model to CPU

model = model.to(torch.device('cpu'))


In [ ]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


which is the best phone that i can buy?) for just $99. And then, with the exception of the base model, only $34 when it comes to hardware. The iPhone 5 is coming in two models, one with $799 in cash and one with $999 in cash, but with both starting at $1,599 in cash and $999 in cash.  The iPhone SE has $999 and $999. The iPhone 12 is coming in two versions with the base model and the entry-level models with the entry-level model with the entry-level model. If you’re new to the line of iPhones and want to upgrade in price, this is your option. If you’ve made it that far in your life and have a budget, this isn’t the phone for you.  Buy your iPhone SE  and the iPhone SE   are the best and the best phones for you in the long run. The  iPhone  is not


In [ ]:
# trainer.save_model('Second_Model_GPT2-Epoch_3_1000')

In [ ]:
# import zipfile
# zip_ref = zipfile.ZipFile("Model_2_GPT-2_Epoch_3_1000.zip", "r")
# zip_ref.extractall()
# zip_ref.close()

In [ ]:
import joblib

# joblib.dump(model, 'SecondModel_Epoch_3_1000_2.joblib')

In [ ]:


joblib.dump(model, 'Model_Epoch_5_3000.joblib')

['Model_Epoch_5_3000.joblib']

In [ ]:
joblib.dump(tokenizer, 'tokenizer_5_3000.joblib')


['tokenizer_5_3000.joblib']

In [ ]:
loaded_model_joblib=joblib.load('SecondModel_Epoch_3_1000_2.joblib')

In [ ]:
loaded_model_joblib

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
loaded_model_joblib

In [ ]:
import joblib as jb

In [ ]:
jb.dump(model,'model_jb.pkl')

In [ ]:
loaded_model = jb.load('model_jb.pkl')

In [ ]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = loaded_model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt').to('cuda'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

#new method

In [ ]:
import joblib as jb

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.6 MB/s eta 0:00:00


In [ ]:
import transformers as tf


In [ ]:
loaded_model=jb.load('/content/drive/MyDrive/Model_Epoch_5_3000.joblib')

In [ ]:
loaded_tokenizer=jb.load('/content/drive/MyDrive/tokenizer_5_3000.joblib')

In [ ]:
loaded_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
loaded_tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})

In [ ]:

# Generate text using the fine-tuned model
prompt = 'what is the use of machine learning'
generated_text = loaded_model.generate(
    input_ids=loaded_tokenizer.encode(prompt, return_tensors='pt'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
    # num_beams=5,
    # early_stopping=True
)

print(loaded_tokenizer.decode(generated_text[0],
                       skip_special_tokens=True
                       ))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


what is the use of machine learning here? You’re talking about an AI program that can do a very deep search of a person’s body, so much like Google. A user, like a doctor, might want to look at a person’s body, or their whole body of their brain. And you see this sort of machine learning that could change and maybe even change where a person is located. How do you find that? It’s kind of like the machine learning of the Internet, which it has a lot of power in that it provides better information than people can use it. It could be a little bit more of a human voice or an electronic medical record. Or it could be in a cell phone, in a cell phone booth. And so on. Now, it’s not a matter of having a data scientist or a person-scientist [at all], but rather, it’s like, in this case, you get


In [ ]:
def generated_text(input):
  input_ids=loaded_tokenizer.encode(input,return_tensors='tf')
  beam_output=loaded_model.generate(input_ids,
                                    max_length=200,
                                    num_beams=5,
                                    no_repeat_ngram_size=2,
                                    early_stopping=True)
  ouput=loaded_tokenizer.decode(beam_output[0],
                                skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
  
  # return '.'.join(output.split('.')[:-1]) + "."
  return output

In [ ]:
def generated_text(input):
  input_ids = loaded_tokenizer.encode(input, return_tensors='pt')
  beam_output = loaded_model.generate(input_ids,
                                      max_length=200,
                                      num_beams=5,
                                      no_repeat_ngram_size=2,
                                      early_stopping=True)
  output = loaded_tokenizer.decode(beam_output[0].tolist(),  # Convert to list
                                   skip_special_tokens=True,
                                   clean_up_tokenization_spaces=True)
  return '.'.join(output.split('.')[:-1]) + "."


In [ ]:
generated_text('what is deep learning ?')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'what is deep learning? It’s a new kind of machine learning that can be applied to a lot of different kinds of data sets,” he said. “It can tell the difference between a human being and a robot. It can also tell a story about a person or a machine. And it can do that in a way that is very human-like. So we think that Deep Learning is going to be a big part of the future of artificial intelligence, because it will be able to tell stories that are not just human, but also machine, and that will have a huge impact on the way people think about data and how they interact with it. We believe that this is the next frontier of AI, so we are excited to work with them to bring it to the forefront of this new era of human interaction and AI in the real world.\nGoogle today  announced  that it has raised a $2.'

In [ ]:
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import gensim.downloader as api

In [ ]:


# Load pre-trained word embeddings
word_vectors =api.load('word2vec-google-news-300')



[=================================================-] 99.9% 1660.8/1662.8MB downloaded


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import torch

In [ ]:
def check(input):
  input_ids = loaded_tokenizer.encode(input, return_tensors='pt')
    
    # Create attention mask
  attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
    
    # Generate text with attention mask and pad token ID
  beam_output = loaded_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=200,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=loaded_tokenizer.pad_token_id,
        eos_token_id=loaded_tokenizer.eos_token_id,
        bos_token_id=loaded_tokenizer.bos_token_id,
        
    )
    
  output = loaded_tokenizer.decode(beam_output[0].tolist(),
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
    
  return '.'.join(output.split('.')[:-1]) + "."


In [ ]:
def new_generated_text(input):

  string1='''Algorithm,Analytics,Android,Antivirus,API,App,Application,Artificial intelligence,Augmented reality,Automation,Backend,Bandwidth,Binary,Bit,Bitcoin,Blockchain,Blog,Bluetooth,Bot,Browser,Cache,Camera,CDN,Chatbot,Circuit,Cloud,CMS,Code,Compiler,Computer,Computing,Cryptography,CSS,Cybersecurity,Data,Database,Debugging,Deep learning,Desktop,DevOps,Device,Digital,Display,Disruptive technology,Domain,Download,E-commerce,Edge computing,Email,Encryption,Enterprise,Event,File,Firmware,Firewall,Framework,Freeware,Frontend,Function,Gateway,Geolocation,GIF,Google,GPU,Graphic design,Grid computing,Growth hacking,Guest post,Gzip,Hardware,Headphones,HTML,HTTP,HTTPS,Hyperlink,Hypertext,IAAS,IDE,Inbound marketing,Index,Information,Infrastructure,Innovation,Input,Interface,Internet,Internet of things,IOS,IP,IP address,IPv4,IPv6,Java,JavaScript,JSON,Keyword,LAMP,LAN,Language,Laptop,Laravel,Layer,Linux,Load balancer,Localhost,Log,Machine learning,Malware,Marketing,Metadata,Microservices,Middleware,Migration,ML,Mobile,Model,Multimedia,MySQL,Native app,Network,Neural network,Nginx,Node.js,Object-oriented programming,Online,Operating system,Optimization,OS,Outsourcing,Overlay,Packet,Password,PC,Performance,Perl,PHP,Platform,Plugin,PM,Podcast,Port,Power bank,Privacy,Processor,Programming,Protocol,Python,Query,Queue,RAM,Ransomware,React,Reboot,Responsive design,REST,Robotics,Ruby,SAAS,Scalability,Scanner,Schema,Script,SDN,Search engine optimization,Security,Server,Service,Shell,SMTP,Software,Source code,Speech recognition,SQL,SSH,SSL,Startup,Storage,Streaming,Subnet,Supercomputer,SVG,System,Tablet,TCP/IP,Technology,Tensorflow,Terminal,Testing,Text editor,Theme,Third-party,Timeline,Traffic,Transmission,Typeface,UI,UNIX,Update,Upload,URL,User,UX,Vector,Version control,Video,Virtual machine,Virtual reality,Virtualization,VPN,Vulnerability,Web,Web design,Web development,Web hosting,Web server,WebGL,Website,Widget,Wireless,WordPress,XML,XSS,YAML,Zend,Zip,ZoomArtificial Intelligence, Machine Learning, Data Science, Big Data, Cloud Computing, Cybersecurity, Internet of Things (IoT), Blockchain, Robotics, Virtual Reality (VR), Augmented Reality (AR), Natural Language Processing (NLP), Quantum Computing, 5G, Edge Computing, DevOps, Agile, Software Engineering, Computer Vision, Deep Learning, Neural Networks, Chatbots, Digital Transformation, Smart Cities, Smart Homes, Industry 4.0, Autonomous Vehicles, Drones, Wearable Technology, Biometric Authentication, Human-Computer Interaction (HCI), User Experience (UX), User Interface (UI), Product Management, Project Management, Agile Development, Scrum, Lean Startup, Growth Hacking, Data Analytics, Business Intelligence (BI), Customer Relationship Management (CRM), Enterprise Resource Planning (ERP), Supply Chain Management, e-Commerce, Digital Marketing, Search Engine Optimization (SEO), Social Media Marketing, Content Marketing, Email Marketing, Affiliate Marketing, Influencer Marketing, Mobile Marketing, Marketing Automation, Customer Acquisition, Customer Retention, Conversion Rate Optimization (CRO), A/B Testing, User Testing, User Research, Design Thinking, User-Centered Design (UCD), Graphic Design, Web Design, Mobile Design, Game Design, Product Design, Interaction Design, Information Architecture, Front-end Development, Back-end Development, Full-stack Development, JavaScript, Python, Java, Ruby, PHP, C++, C#, Swift, Kotlin, TypeScript, HTML, CSS, React, Angular, Vue.js, Node.js, Express.js, Django, Flask, Spring, Ruby on Rails, Laravel, WordPress, Magento, Shopify, WooCommerce, Amazon Web Services (AWS), Microsoft Azure, Google Cloud Platform, Firebase, Heroku, DigitalOcean, GitHub, GitLab, Bitbucket, Jenkins, Docker, Kubernetes, Ansible, Terraform, Elasticsearch, Logstash, Kibana, Grafana, Prometheus, Nagios, Splunk, Zabbix, New Relic, Sumo Logic, Apache Kafka, Apache Spark, Hadoop, MongoDB, MySQL, PostgreSQL, Oracle, Microsoft SQL Server, Redis, Cassandra, Couchbase, Neo4j, Elasticsearch, Solr, Lucene, Nginx, Apache HTTP Server, IIS, Apache Tomcat, Redis, Ansible Tower, Ansible Engine, Ansible Workbench, Ansible Galaxy, Ansible Core, Puppet, Chef, Salt, Vagrant, Consul, Nomad, Packer, OpenStack, Kubernetes-native, Serverless, Docker Swarm, Amazon ECS, Google Kubernetes Engine, Azure Container Service, Microsoft Azure Container Instances, Istio, Knative, OpenShift, Mesosphere, DC/OS, Fargate, Lambda, Azure Functions, Google Cloud Functions, OpenFaaS, Kubeless, AWS Lambda, Serverless Framework, AWS AppSync, GraphQL, Hasura, Prisma, Apollo, Nuxt.js, Gatsby, Next.js, Storybook, Jest, Cypress, Puppeteer, Selenium, Playwright, React Native, Ionic, Flutter, SwiftUI, Kotlin Multiplatform, Mobile App Development, MAlgorithm,Analytics,Android,Antivirus,API,App,Application,Artificial intelligence,Augmented reality,Automation,Backend,Bandwidth,Binary,Bit,Bitcoin,Blockchain,Blog,Bluetooth,Bot,Browser,Cache,Camera,CDN,Chatbot,Circuit,Cloud,CMS,Code,Compiler,Computer,Computing,Cryptography,CSS,Cybersecurity,Data,Database,Debugging,Deep learning,Desktop,DevOps,Device,Digital,Display,Disruptive technology,Domain,Download,E-commerce,Edge computing,Email,Encryption,Enterprise,Event,File,Firmware,Firewall,Framework,Freeware,Frontend,Function,Gateway,Geolocation,GIF,Google,GPU,Graphic design,Grid computing,Growth hacking,Guest post,Gzip,Hardware,Headphones,HTML,HTTP,HTTPS,Hyperlink,Hypertext,IAAS,IDE,Inbound marketing,Index,Information,Infrastructure,Innovation,Input,Interface,Internet,Internet of things,IOS,IP,IP address,IPv4,IPv6,Java,JavaScript,JSON,Keyword,LAMP,LAN,Language,Laptop,Laravel,Layer,Linux,Load balancer,Localhost,Log,Machine learning,Malware,Marketing,Metadata,Microservices,Middleware,Migration,ML,Mobile,Model,Multimedia,MySQL,Native app,Network,Neural network,Nginx,Node.js,Object-oriented programming,Online,Operating system,Optimization,OS,Outsourcing,Overlay,Packet,Password,PC,Performance,Perl,PHP,Platform,Plugin,PM,Podcast,Port,Power bank,Privacy,Processor,Programming,Protocol,Python,Query,Queue,RAM,Ransomware,React,Reboot,Responsive design,REST,Robotics,Ruby,SAAS,Scalability,Scanner,Schema,Script,SDN,Search engine optimization,Security,Server,Service,Shell,SMTP,Software,Source code,Speech recognition,SQL,SSH,SSL,Startup,Storage,Streaming,Subnet,Supercomputer,SVG,System,Tablet,TCP/IP,Technology,Tensorflow,Terminal,Testing,Text editor,Theme,Third-party,Timeline,Traffic,Transmission,Typeface,UI,UNIX,Update,Upload,URL,User,UX,Vector,Version control,Video,Virtual machine,Virtual reality,Virtualization,VPN,Vulnerability,Web,Web design,Web development,Web hosting,Web server,WebGL,Website,Widget,WirelesAlgorithm,Analytics,Android,Antivirus,API,App,Application,Artificial intelligence,Augmented reality,Automation,Backend,Bandwidth,Binary,Bit,Bitcoin,Blockchain,Blog,Bluetooth,Bot,Browser,Cache,Camera,CDN,Chatbot,Circuit,Cloud,CMS,Code,Compiler,Computer,Computing,Cryptography,CSS,Cybersecurity,Data,Database,Debugging,Deep learning,Desktop,DevOps,Device,Digital,Display,Disruptive technology,Domain,Download,E-commerce,Edge computing,Email,Encryption,Enterprise,Event,File,Firmware,Firewall,Framework,Freeware,Frontend,Function,Gateway,Geolocation,GIF,Google,GPU,Graphic design,Grid computing,Growth hacking,Guest post,Gzip,Hardware,Headphones,HTML,HTTP,HTTPS,Hyperlink,Hypertext,IAAS,IDE,Inbound marketing,Index,Information,Infrastructure,Innovation,Input,Interface,Internet,Internet of things,IOS,IP,IP address,IPv4,IPv6,Java,JavaScript,JSON,Keyword,LAMP,LAN,Language,Laptop,Laravel,Layer,Linux,Load balancer,Localhost,Log,Machine learning,Malware,Marketing,Metadata,Microservices,Middleware,Migration,ML,Mobile,Model,Multimedia,MySQL,Native app,Network,Neural network,Nginx,Node.js,Object-oriented programming,Online,Operating system,Optimization,OS,Outsourcing,Overlay,Packet,Password,PC,Performance,Perl,PHP,Platform,Plugin,PM,Podcast,Port,Power bank,Privacy,Processor,Programming,Protocol,Python,Query,Queue,RAM,Ransomware,React,Reboot,Responsive design,REST,Robotics,Ruby,SAAS,Scalability,Scanner,Schema,Script,SDN,Search engine optimization,Security,Server,Service,Shell,SMTP,Software,Source code,Speech recognition,SQL,SSH,SSL,Startup,Storage,Streaming,Subnet,Supercomputer,SVG,System,Tablet,TCP/IP,Technology,Tensorflow,Terminal,Testing,Text editor,Theme,Third-party,Timeline,Traffic,Transmission,Typeface,UI,UNIX,Update,Upload,URL,User,UX,Vector,Version control,Video,Virtual machine,Virtual reality,Virtualization,VPN,Vulnerability,Web,Web design,Web development,Web hosting,Web server,WebGL,Website,Widget,Wireless,WordPress,XML,XSS,YAML,Zend,Zip,Zooms,WordPress,XML,XSS,YAML,Zend,Zip,Zoomobile App Design, Mobile App Marketing, Mobile App Monetization, Mobile App Analytics, Mobile App User Acquisition, Mobile App User Retention, Mobile App User Engagement, Mobile App Push Notifications, Mobile App Personalization, Mobile App A/B Testing, Mobile App User Experience (UX), Mobile App User Interface (UI), Mobile App Accessibility, Mobile App Performance Optimization, Mobile App Security, Mobile App Privacy, Mobile App Data Analytics, Mobile App Business Intelligence, Mobile App CRM, Mobile App Customer Support, Mobile App Localization, Mobile App Internationalization, Mobile App Store Optimization (ASO), Mobile App Promotion, Mobile App Influencer,Apple, Samsung, Google Pixel, OnePlus, Xiaomi, Huawei, Oppo, Vivo, Realme, Sony Xperia, LG, Motorola, Nokia, HTC, BlackBerry, Asus Zenfone, Lenovo, ZTE, Meizu, Honor, Micromax, Karbonn, Lava, Intex, Gionee, Coolpad, Infinix, Tecno, Itel, Swipe, LYF, Jio, Reliance, Vivo, Panasonic, Philips, Sharp, Toshiba, Acer, Dell, HP, Lenovo ThinkPad, Microsoft Surface, Razer Phone, Redmi, Poco, Micromax Canvas, Yu, Asus ROG, Essential Phone, Alcatel, CAT, Black Shark, Elephone, Nokia Lumia, Mobiistar, Google Nexus, Google Pixel, LeEco, LeTV, Meizu, Motorola Moto, Nextbit Robin, Nvidia Shield, Obi Worldphone, OnePlus One, Oppo R, Samsung Galaxy, Sony Xperia, Ulefone'''
  string1=string1.replace(',','')
  tokens1 = word_tokenize(string1.lower())
  tokens2 = word_tokenize(input.lower())

# Convert tokens to vectors
  vector1 = np.mean([word_vectors[token] for token in tokens1 if token in word_vectors], axis=0)
  vector2 = np.mean([word_vectors[token] for token in tokens2 if token in word_vectors], axis=0)

  
# Calculate cosine similarity between the two vectors
  similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
  print(f'the similarity score is {similarity}')
  answer='As an AI langauge model, I have been trained to generate output that is only related to Technology domain. Therefore, I would not be able to answer your question.'
  
  if input in string1 or input in string1.lower():
    return check(input)
  elif similarity<=0.35:
    return answer
  else:
    return check(input)
    

  

In [ ]:
new_generated_text('f')

the similarity score is 0.15153177082538605


'As an AI langauge model, I have been trained to generate output that is only related to Technology domain. Therefore, I would not be able to answer your question.'

In [ ]:
  string1='''Artificial Intelligence, Machine Learning, Data Science, Big Data, Cloud Computing, Cybersecurity, Internet of Things (IoT), Blockchain, Robotics, Virtual Reality (VR), Augmented Reality (AR), Natural Language Processing (NLP), Quantum Computing, 5G, Edge Computing, DevOps, Agile, Software Engineering, Computer Vision, Deep Learning, Neural Networks, Chatbots, Digital Transformation, Smart Cities, Smart Homes, Industry 4.0, Autonomous Vehicles, Drones, Wearable Technology, Biometric Authentication, Human-Computer Interaction (HCI), User Experience (UX), User Interface (UI), Product Management, Project Management, Agile Development, Scrum, Lean Startup, Growth Hacking, Data Analytics, Business Intelligence (BI), Customer Relationship Management (CRM), Enterprise Resource Planning (ERP), Supply Chain Management, e-Commerce, Digital Marketing, Search Engine Optimization (SEO), Social Media Marketing, Content Marketing, Email Marketing, Affiliate Marketing, Influencer Marketing, Mobile Marketing, Marketing Automation, Customer Acquisition, Customer Retention, Conversion Rate Optimization (CRO), A/B Testing, User Testing, User Research, Design Thinking, User-Centered Design (UCD), Graphic Design, Web Design, Mobile Design, Game Design, Product Design, Interaction Design, Information Architecture, Front-end Development, Back-end Development, Full-stack Development, JavaScript, Python, Java, Ruby, PHP, C++, C#, Swift, Kotlin, TypeScript, HTML, CSS, React, Angular, Vue.js, Node.js, Express.js, Django, Flask, Spring, Ruby on Rails, Laravel, WordPress, Magento, Shopify, WooCommerce, Amazon Web Services (AWS), Microsoft Azure, Google Cloud Platform, Firebase, Heroku, DigitalOcean, GitHub, GitLab, Bitbucket, Jenkins, Docker, Kubernetes, Ansible, Terraform, Elasticsearch, Logstash, Kibana, Grafana, Prometheus, Nagios, Splunk, Zabbix, New Relic, Sumo Logic, Apache Kafka, Apache Spark, Hadoop, MongoDB, MySQL, PostgreSQL, Oracle, Microsoft SQL Server, Redis, Cassandra, Couchbase, Neo4j, Elasticsearch, Solr, Lucene, Nginx, Apache HTTP Server, IIS, Apache Tomcat, Redis, Ansible Tower, Ansible Engine, Ansible Workbench, Ansible Galaxy, Ansible Core, Puppet, Chef, Salt, Vagrant, Consul, Nomad, Packer, OpenStack, Kubernetes-native, Serverless, Docker Swarm, Amazon ECS, Google Kubernetes Engine, Azure Container Service, Microsoft Azure Container Instances, Istio, Knative, OpenShift, Mesosphere, DC/OS, Fargate, Lambda, Azure Functions, Google Cloud Functions, OpenFaaS, Kubeless, AWS Lambda, Serverless Framework, AWS AppSync, GraphQL, Hasura, Prisma, Apollo, Nuxt.js, Gatsby, Next.js, Storybook, Jest, Cypress, Puppeteer, Selenium, Playwright, React Native, Ionic, Flutter, SwiftUI, Kotlin Multiplatform, Mobile App Development, MAlgorithm,Analytics,Android,Antivirus,API,App,Application,Artificial intelligence,Augmented reality,Automation,Backend,Bandwidth,Binary,Bit,Bitcoin,Blockchain,Blog,Bluetooth,Bot,Browser,Cache,Camera,CDN,Chatbot,Circuit,Cloud,CMS,Code,Compiler,Computer,Computing,Cryptography,CSS,Cybersecurity,Data,Database,Debugging,Deep learning,Desktop,DevOps,Device,Digital,Display,Disruptive technology,Domain,Download,E-commerce,Edge computing,Email,Encryption,Enterprise,Event,File,Firmware,Firewall,Framework,Freeware,Frontend,Function,Gateway,Geolocation,GIF,Google,GPU,Graphic design,Grid computing,Growth hacking,Guest post,Gzip,Hardware,Headphones,HTML,HTTP,HTTPS,Hyperlink,Hypertext,IAAS,IDE,Inbound marketing,Index,Information,Infrastructure,Innovation,Input,Interface,Internet,Internet of things,IOS,IP,IP address,IPv4,IPv6,Java,JavaScript,JSON,Keyword,LAMP,LAN,Language,Laptop,Laravel,Layer,Linux,Load balancer,Localhost,Log,Machine learning,Malware,Marketing,Metadata,Microservices,Middleware,Migration,ML,Mobile,Model,Multimedia,MySQL,Native app,Network,Neural network,Nginx,Node.js,Object-oriented programming,Online,Operating system,Optimization,OS,Outsourcing,Overlay,Packet,Password,PC,Performance,Perl,PHP,Platform,Plugin,PM,Podcast,Port,Power bank,Privacy,Processor,Programming,Protocol,Python,Query,Queue,RAM,Ransomware,React,Reboot,Responsive design,REST,Robotics,Ruby,SAAS,Scalability,Scanner,Schema,Script,SDN,Search engine optimization,Security,Server,Service,Shell,SMTP,Software,Source code,Speech recognition,SQL,SSH,SSL,Startup,Storage,Streaming,Subnet,Supercomputer,SVG,System,Tablet,TCP/IP,Technology,Tensorflow,Terminal,Testing,Text editor,Theme,Third-party,Timeline,Traffic,Transmission,Typeface,UI,UNIX,Update,Upload,URL,User,UX,Vector,Version control,Video,Virtual machine,Virtual reality,Virtualization,VPN,Vulnerability,Web,Web design,Web development,Web hosting,Web server,WebGL,Website,Widget,WirelesAlgorithm,Analytics,Android,Antivirus,API,App,Application,Artificial intelligence,Augmented reality,Automation,Backend,Bandwidth,Binary,Bit,Bitcoin,Blockchain,Blog,Bluetooth,Bot,Browser,Cache,Camera,CDN,Chatbot,Circuit,Cloud,CMS,Code,Compiler,Computer,Computing,Cryptography,CSS,Cybersecurity,Data,Database,Debugging,Deep learning,Desktop,DevOps,Device,Digital,Display,Disruptive technology,Domain,Download,E-commerce,Edge computing,Email,Encryption,Enterprise,Event,File,Firmware,Firewall,Framework,Freeware,Frontend,Function,Gateway,Geolocation,GIF,Google,GPU,Graphic design,Grid computing,Growth hacking,Guest post,Gzip,Hardware,Headphones,HTML,HTTP,HTTPS,Hyperlink,Hypertext,IAAS,IDE,Inbound marketing,Index,Information,Infrastructure,Innovation,Input,Interface,Internet,Internet of things,IOS,IP,IP address,IPv4,IPv6,Java,JavaScript,JSON,Keyword,LAMP,LAN,Language,Laptop,Laravel,Layer,Linux,Load balancer,Localhost,Log,Machine learning,Malware,Marketing,Metadata,Microservices,Middleware,Migration,ML,Mobile,Model,Multimedia,MySQL,Native app,Network,Neural network,Nginx,Node.js,Object-oriented programming,Online,Operating system,Optimization,OS,Outsourcing,Overlay,Packet,Password,PC,Performance,Perl,PHP,Platform,Plugin,PM,Podcast,Port,Power bank,Privacy,Processor,Programming,Protocol,Python,Query,Queue,RAM,Ransomware,React,Reboot,Responsive design,REST,Robotics,Ruby,SAAS,Scalability,Scanner,Schema,Script,SDN,Search engine optimization,Security,Server,Service,Shell,SMTP,Software,Source code,Speech recognition,SQL,SSH,SSL,Startup,Storage,Streaming,Subnet,Supercomputer,SVG,System,Tablet,TCP/IP,Technology,Tensorflow,Terminal,Testing,Text editor,Theme,Third-party,Timeline,Traffic,Transmission,Typeface,UI,UNIX,Update,Upload,URL,User,UX,Vector,Version control,Video,Virtual machine,Virtual reality,Virtualization,VPN,Vulnerability,Web,Web design,Web development,Web hosting,Web server,WebGL,Website,Widget,Wireless,WordPress,XML,XSS,YAML,Zend,Zip,Zooms,WordPress,XML,XSS,YAML,Zend,Zip,Zoomobile App Design, Mobile App Marketing, Mobile App Monetization, Mobile App Analytics, Mobile App User Acquisition, Mobile App User Retention, Mobile App User Engagement, Mobile App Push Notifications, Mobile App Personalization, Mobile App A/B Testing, Mobile App User Experience (UX), Mobile App User Interface (UI), Mobile App Accessibility, Mobile App Performance Optimization, Mobile App Security, Mobile App Privacy, Mobile App Data Analytics, Mobile App Business Intelligence, Mobile App CRM, Mobile App Customer Support, Mobile App Localization, Mobile App Internationalization, Mobile App Store Optimization (ASO), Mobile App Promotion, Mobile App Influencer,Apple, Samsung, Google Pixel, OnePlus, Xiaomi, Huawei, Oppo, Vivo, Realme, Sony Xperia, LG, Motorola, Nokia, HTC, BlackBerry, Asus Zenfone, Lenovo, ZTE, Meizu, Honor, Micromax, Karbonn, Lava, Intex, Gionee, Coolpad, Infinix, Tecno, Itel, Swipe, LYF, Jio, Reliance, Vivo, Panasonic, Philips, Sharp, Toshiba, Acer, Dell, HP, Lenovo ThinkPad, Microsoft Surface, Razer Phone, Redmi, Poco, Micromax Canvas, Yu, Asus ROG, Essential Phone, Alcatel, CAT, Black Shark, Elephone, Nokia Lumia, Mobiistar, Google Nexus, Google Pixel, LeEco, LeTV, Meizu, Motorola Moto, Nextbit Robin, Nvidia Shield, Obi Worldphone, OnePlus One, Oppo R, Samsung Galaxy, Sony Xperia, Ulefone, Vodafone Smart, Xiaomi Mi, ZTE Nubia.   '''


In [ ]:
def is_Tech_art(s2):
  # Tokenize the strings
  tokens1 = word_tokenize(string1.lower())
  tokens2 = word_tokenize(s2)

# Convert tokens to vectors
  vector1 = np.mean([word_vectors[token] for token in tokens1 if token in word_vectors], axis=0)
  vector2 = np.mean([word_vectors[token] for token in tokens2 if token in word_vectors], axis=0)

# Calculate cosine similarity between the two vectors
  similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
  print(similarity)
  if similarity >= 0.55:
    return 1
  else:
    return 0



In [ ]:
is_Tech_art('give me some knowledge about finance')

0.30888936


0

In [ ]:
# def is_Tech_art(s2):
#   # Tokenize the strings
#   string1='''Artificial Intelligence, Machine Learning, Data Science, Big Data, Cloud Computing, Cybersecurity, Internet of Things (IoT), Blockchain, Robotics, Virtual Reality (VR), Augmented Reality (AR), Natural Language Processing (NLP), Quantum Computing, 5G, Edge Computing, DevOps, Agile, Software Engineering, Computer Vision, Deep Learning, Neural Networks, Chatbots, Digital Transformation, Smart Cities, Smart Homes, Industry 4.0, Autonomous Vehicles, Drones, Wearable Technology, Biometric Authentication, Human-Computer Interaction (HCI), User Experience (UX), User Interface (UI), Product Management, Project Management, Agile Development, Scrum, Lean Startup, Growth Hacking, Data Analytics, Business Intelligence (BI), Customer Relationship Management (CRM), Enterprise Resource Planning (ERP), Supply Chain Management, e-Commerce, Digital Marketing, Search Engine Optimization (SEO), Social Media Marketing, Content Marketing, Email Marketing, Affiliate Marketing, Influencer Marketing, Mobile Marketing, Marketing Automation, Customer Acquisition, Customer Retention, Conversion Rate Optimization (CRO), A/B Testing, User Testing, User Research, Design Thinking, User-Centered Design (UCD), Graphic Design, Web Design, Mobile Design, Game Design, Product Design, Interaction Design, Information Architecture, Front-end Development, Back-end Development, Full-stack Development, JavaScript, Python, Java, Ruby, PHP, C++, C#, Swift, Kotlin, TypeScript, HTML, CSS, React, Angular, Vue.js, Node.js, Express.js, Django, Flask, Spring, Ruby on Rails, Laravel, WordPress, Magento, Shopify, WooCommerce, Amazon Web Services (AWS), Microsoft Azure, Google Cloud Platform, Firebase, Heroku, DigitalOcean, GitHub, GitLab, Bitbucket, Jenkins, Docker, Kubernetes, Ansible, Terraform, Elasticsearch, Logstash, Kibana, Grafana, Prometheus, Nagios, Splunk, Zabbix, New Relic, Sumo Logic, Apache Kafka, Apache Spark, Hadoop, MongoDB, MySQL, PostgreSQL, Oracle, Microsoft SQL Server, Redis, Cassandra, Couchbase, Neo4j, Elasticsearch, Solr, Lucene, Nginx, Apache HTTP Server, IIS, Apache Tomcat, Redis, Ansible Tower, Ansible Engine, Ansible Workbench, Ansible Galaxy, Ansible Core, Puppet, Chef, Salt, Vagrant, Consul, Nomad, Packer, OpenStack, Kubernetes-native, Serverless, Docker Swarm, Amazon ECS, Google Kubernetes Engine, Azure Container Service, Microsoft Azure Container Instances, Istio, Knative, OpenShift, Mesosphere, DC/OS, Fargate, Lambda, Azure Functions, Google Cloud Functions, OpenFaaS, Kubeless, AWS Lambda, Serverless Framework, AWS AppSync, GraphQL, Hasura, Prisma, Apollo, Nuxt.js, Gatsby, Next.js, Storybook, Jest, Cypress, Puppeteer, Selenium, Playwright, React Native, Ionic, Flutter, SwiftUI, Kotlin Multiplatform, Mobile App Development, MAlgorithm,Analytics,Android,Antivirus,API,App,Application,Artificial intelligence,Augmented reality,Automation,Backend,Bandwidth,Binary,Bit,Bitcoin,Blockchain,Blog,Bluetooth,Bot,Browser,Cache,Camera,CDN,Chatbot,Circuit,Cloud,CMS,Code,Compiler,Computer,Computing,Cryptography,CSS,Cybersecurity,Data,Database,Debugging,Deep learning,Desktop,DevOps,Device,Digital,Display,Disruptive technology,Domain,Download,E-commerce,Edge computing,Email,Encryption,Enterprise,Event,File,Firmware,Firewall,Framework,Freeware,Frontend,Function,Gateway,Geolocation,GIF,Google,GPU,Graphic design,Grid computing,Growth hacking,Guest post,Gzip,Hardware,Headphones,HTML,HTTP,HTTPS,Hyperlink,Hypertext,IAAS,IDE,Inbound marketing,Index,Information,Infrastructure,Innovation,Input,Interface,Internet,Internet of things,IOS,IP,IP address,IPv4,IPv6,Java,JavaScript,JSON,Keyword,LAMP,LAN,Language,Laptop,Laravel,Layer,Linux,Load balancer,Localhost,Log,Machine learning,Malware,Marketing,Metadata,Microservices,Middleware,Migration,ML,Mobile,Model,Multimedia,MySQL,Native app,Network,Neural network,Nginx,Node.js,Object-oriented programming,Online,Operating system,Optimization,OS,Outsourcing,Overlay,Packet,Password,PC,Performance,Perl,PHP,Platform,Plugin,PM,Podcast,Port,Power bank,Privacy,Processor,Programming,Protocol,Python,Query,Queue,RAM,Ransomware,React,Reboot,Responsive design,REST,Robotics,Ruby,SAAS,Scalability,Scanner,Schema,Script,SDN,Search engine optimization,Security,Server,Service,Shell,SMTP,Software,Source code,Speech recognition,SQL,SSH,SSL,Startup,Storage,Streaming,Subnet,Supercomputer,SVG,System,Tablet,TCP/IP,Technology,Tensorflow,Terminal,Testing,Text editor,Theme,Third-party,Timeline,Traffic,Transmission,Typeface,UI,UNIX,Update,Upload,URL,User,UX,Vector,Version control,Video,Virtual machine,Virtual reality,Virtualization,VPN,Vulnerability,Web,Web design,Web development,Web hosting,Web server,WebGL,Website,Widget,WirelesAlgorithm,Analytics,Android,Antivirus,API,App,Application,Artificial intelligence,Augmented reality,Automation,Backend,Bandwidth,Binary,Bit,Bitcoin,Blockchain,Blog,Bluetooth,Bot,Browser,Cache,Camera,CDN,Chatbot,Circuit,Cloud,CMS,Code,Compiler,Computer,Computing,Cryptography,CSS,Cybersecurity,Data,Database,Debugging,Deep learning,Desktop,DevOps,Device,Digital,Display,Disruptive technology,Domain,Download,E-commerce,Edge computing,Email,Encryption,Enterprise,Event,File,Firmware,Firewall,Framework,Freeware,Frontend,Function,Gateway,Geolocation,GIF,Google,GPU,Graphic design,Grid computing,Growth hacking,Guest post,Gzip,Hardware,Headphones,HTML,HTTP,HTTPS,Hyperlink,Hypertext,IAAS,IDE,Inbound marketing,Index,Information,Infrastructure,Innovation,Input,Interface,Internet,Internet of things,IOS,IP,IP address,IPv4,IPv6,Java,JavaScript,JSON,Keyword,LAMP,LAN,Language,Laptop,Laravel,Layer,Linux,Load balancer,Localhost,Log,Machine learning,Malware,Marketing,Metadata,Microservices,Middleware,Migration,ML,Mobile,Model,Multimedia,MySQL,Native app,Network,Neural network,Nginx,Node.js,Object-oriented programming,Online,Operating system,Optimization,OS,Outsourcing,Overlay,Packet,Password,PC,Performance,Perl,PHP,Platform,Plugin,PM,Podcast,Port,Power bank,Privacy,Processor,Programming,Protocol,Python,Query,Queue,RAM,Ransomware,React,Reboot,Responsive design,REST,Robotics,Ruby,SAAS,Scalability,Scanner,Schema,Script,SDN,Search engine optimization,Security,Server,Service,Shell,SMTP,Software,Source code,Speech recognition,SQL,SSH,SSL,Startup,Storage,Streaming,Subnet,Supercomputer,SVG,System,Tablet,TCP/IP,Technology,Tensorflow,Terminal,Testing,Text editor,Theme,Third-party,Timeline,Traffic,Transmission,Typeface,UI,UNIX,Update,Upload,URL,User,UX,Vector,Version control,Video,Virtual machine,Virtual reality,Virtualization,VPN,Vulnerability,Web,Web design,Web development,Web hosting,Web server,WebGL,Website,Widget,Wireless,WordPress,XML,XSS,YAML,Zend,Zip,Zooms,WordPress,XML,XSS,YAML,Zend,Zip,Zoomobile App Design, Mobile App Marketing, Mobile App Monetization, Mobile App Analytics, Mobile App User Acquisition, Mobile App User Retention, Mobile App User Engagement, Mobile App Push Notifications, Mobile App Personalization, Mobile App A/B Testing, Mobile App User Experience (UX), Mobile App User Interface (UI), Mobile App Accessibility, Mobile App Performance Optimization, Mobile App Security, Mobile App Privacy, Mobile App Data Analytics, Mobile App Business Intelligence, Mobile App CRM, Mobile App Customer Support, Mobile App Localization, Mobile App Internationalization, Mobile App Store Optimization (ASO), Mobile App Promotion, Mobile App Influencer,Apple, Samsung, Google Pixel, OnePlus, Xiaomi, Huawei, Oppo, Vivo, Realme, Sony Xperia, LG, Motorola, Nokia, HTC, BlackBerry, Asus Zenfone, Lenovo, ZTE, Meizu, Honor, Micromax, Karbonn, Lava, Intex, Gionee, Coolpad, Infinix, Tecno, Itel, Swipe, LYF, Jio, Reliance, Vivo, Panasonic, Philips, Sharp, Toshiba, Acer, Dell, HP, Lenovo ThinkPad, Microsoft Surface, Razer Phone, Redmi, Poco, Micromax Canvas, Yu, Asus ROG, Essential Phone, Alcatel, CAT, Black Shark, Elephone, Nokia Lumia, Mobiistar, Google Nexus, Google Pixel, LeEco, LeTV, Meizu, Motorola Moto, Nextbit Robin, Nvidia Shield, Obi Worldphone, OnePlus One, Oppo R, Samsung Galaxy, Sony Xperia, Ulefone, Vodafone Smart, Xiaomi Mi, ZTE Nubia.   '''
#   string1=string1.replace(',','')
#   tokens1 = word_tokenize(string1.lower())
#   tokens2 = word_tokenize(s2.lower())

# # Convert tokens to vectors
#   vector1 = np.mean([word_vectors[token] for token in tokens1 if token in word_vectors], axis=0)
#   vector2 = np.mean([word_vectors[token] for token in tokens2 if token in word_vectors], axis=0)

# # Calculate cosine similarity between the two vectors
#   similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
#   # if similarity >= 0.55:
#   #   return 1
#   # else:
#   #   return 0
#   if similarity<=0.30:
#     print('As an AI langauge model, I have been trained to generate output that is only related to Technology domain. Therefore, I would not be able to answer your question.')
#   else if s2 in string1 or s2 in string1.lower():
#     return 
#     # return '.'.join(output.split('.')[:-1]) + "."
#   else :
#     return '.'.join(output.split('.')[:-1]) + "."

  



In [ ]:
import torch

In [ ]:
new_generated_text('what is deep learning ?')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'what is deep learning? It’s a new kind of machine learning that can be applied to a lot of different kinds of data sets,” he said. “It can tell the difference between a human being and a robot. It can also tell a story about a person or a machine. And it can do that in a way that is very human-like. So we think that Deep Learning is going to be a big part of the future of artificial intelligence, because it will be able to tell stories that are not just human, but also machine, and that will have a huge impact on the way people think about data and how they interact with it. We believe that this is the next frontier of AI, so we are excited to work with them to bring it to the forefront of this new era of human interaction and AI in the real world.\nGoogle today  announced  that it has raised a $2.'

In [ ]:
def my_function():
    a = 'idf si fds'
    if a != 0:
        return a
        
my_function()


'idf si fds'

In [ ]:
import spacy

In [ ]:
# !pip install -U spacy
!python -m spacy download en_core_web_sm


In [ ]:
nlp=spacy.load('en_core_web_lg')

# Checking similarity

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

import spacy
import pandas as pd



In [ ]:
# Load the spaCy English language model
nlp = spacy.load('en_core_web_md')




In [ ]:
# Load the dataset into a pandas dataframe
df = pd.read_csv('/content/drive/MyDrive/data.csv')


In [ ]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


In [ ]:
df['article'].astype(str)

0        European Union lawmakers are considering wheth...
1        Industrial robots are expensive. But, then, so...
2        Roblox announced today that it’s buying a digi...
3        Just six month after raising its first bit of ...
4        Spotify today announced the global launch of v...
                               ...                        
25294    Volkswagen has started to sell a home-charging...
25295    The 2020 Ford Escape plug-in hybrid — a first ...
25296    Uber has appointed Pradeep Parameswaran, who o...
25297    Spin has launched its scooter sharing business...
25298    While EU lawmakers are mulling a temporary ban...
Name: article, Length: 25299, dtype: object

pandas.core.series.Series

In [ ]:
import numpy as np

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"
...,...,...,...,...,...,...
25294,{'$oid': '6010dd3d4bacf659791a0081'},Volkswagen launches home EV charging system sa...,2020-06-12 11:35:51,Volkswagen has started to sell a home-charging...,Volkswagen has started to sell a home-charging...,"automotive industry, charging stations, electr..."
25295,{'$oid': '6010dd3d4bacf659791a0084'},Ford’s 2020 Escape plug-in hybrid takes aim at...,2020-06-08 09:54:34,The 2020 Ford Escape plug-in hybrid — a first ...,The 2020 Ford Escape plug-in hybrid — a first ...,"electric vehicles, Environmental Protection Ag..."
25296,{'$oid': '6010dd434bacf659791a009e'},Uber appoints Pradeep Parameswaran as new head...,2020-06-10 20:47:36,"Uber has appointed Pradeep Parameswaran, who o...","Uber has appointed Pradeep Parameswaran, who o...","asia pacific, China, Dara Khosrowshahi, india,..."
25297,{'$oid': '6010dd4b4bacf659791a00c6'},"Spin scooters head to Europe, starting with Ge...",2020-06-10 11:52:00,Spin has launched its scooter sharing business...,Spin has launched its scooter sharing business...,"amsterdam, Atlanta, bird, Circ, cologne, Derri..."


In [ ]:
my_string=df['article'][2001:5000]
my_series=pd.Series(my_string)

In [ ]:
my_series

0       European Union lawmakers are considering wheth...
1       Industrial robots are expensive. But, then, so...
2       Roblox announced today that it’s buying a digi...
3       Just six month after raising its first bit of ...
4       Spotify today announced the global launch of v...
                              ...                        
1011    Chinese giant ByteDance is engaging in early d...
1012    Everyone wants more bandwidth from the skies, ...
1013    As it reached an estimated valuation of  $100 ...
1014    SpaceX and NASA have made history once again, ...
1015    I wanted to write  an essay about  Microsoft a...
Name: article, Length: 1000, dtype: object

In [ ]:
def vectorize_text(text):
    if isinstance(text, float) and np.isnan(text):
        return np.zeros(300)  # replace NaN values with a vector of zeros
    else:
        doc = nlp(text)   
        return doc.vector

# Apply the vectorize_text function to each row of the 'articles' column

df['article_vectors'] = my_series.apply(vectorize_text)


In [ ]:
type(df['article'])

pandas.core.series.Series

In [ ]:
df[:5000]

,_id,title,datePublished,description,article,keywords,article_vectors
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream...",NaN
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct...",NaN
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ...",NaN
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par...",NaN
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube",NaN
...,...,...,...,...,...,...,...
5165,{'$oid': '60085289b5425d67582ec44a'},"Bicycle Health, the virtual opioid use disorde...",2020-12-16 06:30:38,The startup opioid use disorder therapy servic...,The startup opioid use disorder therapy servic...,"Bicycle Health, california, Disease, drugs, Ge...","[-1.744421, 0.5546731, -1.8126562, 0.62611115,..."
5166,{'$oid': '60085289b5425d67582ec44b'},Amazon asks judge to set aside Microsoft’s $10...,2020-12-16 09:06:45,It’s been more than two years since the Pentag...,It’s been more than two years since the Pentag...,"Amazon, Enterprise, Microsoft, Pentagon JEDI c...","[-1.7384604, 0.49680993, -2.2390392, 0.3426388..."
5167,{'$oid': '60085289b5425d67582ec44c'},"Google, Cisco and VMware join Microsoft to opp...",2020-12-21 15:23:42,Silicon Valley tech giants say spyware maker N...,A coalition of companies have filed an amicus...,"Cisco, computer security, computing, espionage...","[-1.6851072, 0.3471673, -1.729075, 0.6245949, ..."
5168,{'$oid': '6008528ab5425d67582ec44d'},Austin-based ReturnSafe raises $3.25 million f...,2020-12-22 08:59:44,"ReturnSafe, a symptom checking and contact tra...","ReturnSafe , a symptom checking and contact tr...","Active Capital, COVID-19, economy, fifty years...","[-1.5034337, 1.024634, -2.486507, 0.81868577, ..."


In [ ]:
df.to_csv('article_vectors_5000.csv',index=False)

In [ ]:
!cp  article_vectors_5000.csv /content/drive/MyDrive/

In [ ]:
df['article_vectors'][0]

In [ ]:
import pandas as pd
# from transformers import pipeline


# Load the DataFrame with the "article" column
df = pd.read_csv("/content/drive/MyDrive/Major_Project/data.csv")

In [ ]:
df.dropna(inplace=True)

In [ ]:
# # Define the text similarity pipeline
# text_similarity = pipeline(
#     "text2text-generation",
#     model="sentence-transformers/paraphrase-mpnet-base-v2",
#     tokenizer="sentence-transformers/paraphrase-mpnet-base-v2"
# )

In [ ]:
# # Get user input prompt
# prompt = input("Enter a prompt: ")

In [ ]:
list=[df['keywords'][i].replace(',','').split()]

['audio-visual', 'eu', 'geoblocking', 'Netflix', 'streaming', 'TV']

In [ ]:
# Concatenate all the articles into a single string
combined_articles = " ".join(df["keywords"])

In [ ]:
combined_articles=combined_articles.replace(',','')

In [ ]:
combined_articles=combined_articles.split()

In [ ]:
combined_articles

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# string = combined_articles.lower()
# words = string.split()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# words

In [ ]:
# filtered_words = [word for word in combined_articles if word not in stopwords.words('english')]

In [ ]:
# filtered_words

In [ ]:
# filtered_string = ' '.join(filtered_words)


In [ ]:
# filtered_string

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Example corpus of documents

# Create the TfidfVectorizer object
vectorizer = TfidfVectorizer()



# Fit the vectorizer to the corpus and transform the corpus
tfidf_matrix = vectorizer.fit_transform([combined_articles])

# Convert the sparse matrix to a dense matrix (if necessary)
dense_matrix = tfidf_matrix.toarray()

# Print the dense matrix
print(dense_matrix)


[[0.00079699 0.00326766 0.00095639 ... 0.00103609 0.00207218 0.0003985 ]]


In [ ]:
# Calculate the total tf-idf scores for each word
total_scores = dense_matrix.sum(axis=0)

In [ ]:
# Get the vocabulary (mapping of feature indices to feature names)
vocabulary = vectorizer.vocabulary_


In [ ]:
# Create a list of (word, score) tuples and sort it by score
word_scores = [(word, total_scores[vocabulary[word]]) for word in vocabulary]
word_scores = sorted(word_scores, key=lambda x: x[1], reverse=True)



In [ ]:
# Print the top 10 most impactful words
list_new=[]
new_string=''
for word, score in word_scores[:5000]:
  list_new.append(word)
  # new_string=new_string + word
  print(f"{word}")

In [ ]:
list_new

In [ ]:
filtered_words = [word for word in list_new if word not in stopwords.words('english')]

In [ ]:
filtered_words

In [ ]:
words_string = ','.join(filtered_words)

print(words_string)

In [ ]:
import re

# use regular expression to remove numbers
updated_string = re.sub(r'\d+', '', words_string)

print(updated_string)  


# Bag of Words

In [ ]:
bag_of_words=updated_string

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_

import spacy
import pandas as pd



In [ ]:
nlp = spacy.load('en_core_web_sm')


In [ ]:

string1 = bag_of_words
string2 = "give a glance about artificial intelligence"

doc1 = nlp(string1)
doc2 = nlp(string2)

similarity_score = doc1.similarity(doc2)

print("Similarity score:", similarity_score)


Similarity score: 0.09441967930980547


<ipython-input-120-0699e0bd6531>:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_score = doc1.similarity(doc2)


In [ ]:

# Load the English language model
# nlp = spacy.load("en_core_web_sm")

# Define the list of words to lemmatize

# Apply lemmatization to the list of words
lemmas = [token.lemma_ for token in nlp(" ".join(list_new))]

# Print the lemmas
print(lemmas)


In [ ]:
def is_Tech_art(s2):
  # Tokenize the strings
  tokens1 = word_tokenize(bag_of_words.lower())
  tokens2 = word_tokenize(s2)

# Convert tokens to vectors
  vector1 = np.mean([word_vectors[token] for token in tokens1 if token in word_vectors], axis=0)
  vector2 = np.mean([word_vectors[token] for token in tokens2 if token in word_vectors], axis=0)

# Calculate cosine similarity between the two vectors
  similarity = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
  print(similarity)



In [ ]:
is_Tech_art('')

0.5087282


In [ ]:
import spacy

# Load a spaCy model with WordNet integration
nlp = spacy.load("en_core_web_sm")



In [ ]:
pip install spacy-wordnet


In [ ]:
nlp=spacy.load('en_core_web_sm')

In [ ]:
nlp.add_pipe("lemmatizer", name="wordnet", after="tagger")


In [ ]:
# Access WordNet via spaCy's Language object
word=nlp.get_pipe("wordnet")

# Use the WordNet integration to retrieve synsets for a word in a specific domain
synsets = word._.wordnet.wordnet_synsets_for_domain("technology")

# synsets = wordnet.wordnet_synsets_for_domain("technology")


In [ ]:
!pip install --upgrade nltk

In [ ]:
import nltk; print(nltk.__version__)

3.8.1


In [ ]:
from nltk.corpus import wordnet_domains

# Find synsets related to the technology domain
tech_synsets = wordnet_domains.synsets('technology')


In [ ]:
!export LC_ALL=en_US.UTF-8


# Input prompt using Synonyms 

Necessary Import


In [ ]:
import spacy
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
prompt=input("Enter a sentence: ")
tokens=nltk.word_tokenize(prompt)
synonyms=[]
# loop through each token and find its synonyms
for token in tokens:
    for syn in wordnet.synsets(token):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())

synonyms

Enter a sentence: computer science


['computer',
 'computing_machine',
 'computing_device',
 'data_processor',
 'electronic_computer',
 'information_processing_system',
 'calculator',
 'reckoner',
 'figurer',
 'estimator',
 'computer',
 'science',
 'scientific_discipline',
 'skill',
 'science']

In [ ]:
# i=0
# for token in tokens:
#   synsets[i] = wordnet.synsets(token)
#   i=i+1

In [ ]:
# filter the synonyms to only include those related to technology
technology_synonyms = ['technology', 'innovation', 'automation', 'digital', 'software','technology', 'innovation', 'automation', 'digital',
                       'software', 'hardware', 'artificial','intelligence', 'machine','learning', 'robotics', 'internet', 'web, data',
                       'analytics', 'cloud, computing', 'virtual', 'reality', 'augmented', 'cybersecurity', 'blockchain', 'cryptocurrency',
                       'mobile', 'app', 'gadget', 'device', 'network', 'communication']
filtered_synonyms = list(set(synonyms).intersection(set(technology_synonyms)))


In [ ]:
filtered_synonyms

['learning', 'machine']

In [ ]:
def input_text(prompt):
  # prompt=input("Enter a sentence: ")
  tokens=nltk.word_tokenize(prompt)
  synonyms=[]
  # loop through each token and find its synonyms
  for token in tokens:
    for syn in wordnet.synsets(token):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())



  # filter the synonyms to only include those related to technology
  technology_synonyms = ['technology', 'innovation', 'automation', 'digital', 'software','technology', 'innovation', 'automation', 'digital',
                       'software', 'hardware', 'artificial','intelligence', 'machine','learning', 'robotics', 'internet', 'web', 'science','data',
                       'analytics', 'cloud, computing', 'virtual', 'reality', 'augmented', 'cybersecurity', 'blockchain', 'cryptocurrency',
                       'mobile', 'app', 'gadget', 'device', 'network', 'communication',' modeling', 'mining',' engineering','statistical',
                       'Computing', 'Cybernetics', 'tech', ' Nanotechnology', 'Robotics', 'Telecommunications', 'Virtual reality','phone','analytics']
  filtered_synonyms = list(set(synonyms).intersection(set(technology_synonyms)))

  print(filtered_synonyms)

  # give an output only if the number of matching words is equal to or greater than one
  if len(filtered_synonyms) >=1:
    print('Output')
  else:
    print('Not Valid')

In [ ]:
input_text('data analytics')

['data']
Output
